# Prototype: Data Visualization

In [1]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import open3d.visualization as vis

import sys
sys.path.insert(0, '../impl/utils/')
import voxel_processing as vp

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


device(type='cuda')

In [2]:
# Load datasets
data = vp.read_dataset_from_path('../data/func_test/voxel/')

Detected 2 dataset(s). 


In [10]:
mesh = o3d.io.read_triangle_mesh("../data/func_test/obj/test.obj")
mesh.compute_vertex_normals()
plane = np.array([0.9538,  0.2334, -0.1813, -0.0546])
import visualization as prsvis
prsvis.visualize_planar_symmetry(mesh, plane)

[visualization.py] Drawing symmetry plane [ 0.9538  0.2334 -0.1813 -0.0546]


In [7]:
pcd = mesh.sample_points_uniformly(number_of_points=1000)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=float(2/32 + 0.001))
prsvis.visualize_planar_symmetry(voxel_grid, plane)

[visualization.py] Drawing symmetry plane [ 0.5513  0.5534 -0.0589 -0.0903 -0.1242]


In [11]:
mesh_box = o3d.geometry.TriangleMesh.create_box(width=1, height=1, depth=0.01)
mesh_box.paint_uniform_color(np.array([0.1, 0.1, 0.4]))
mesh_box.translate(np.array([-.5, -.5, 0.])) 
frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
frame2 = o3d.geometry.TriangleMesh.create_coordinate_frame()

def get_trans_mat(normal):
    z = normal
    y = z - np.array([0., 0., 0.3])
    y = y - np.dot(y, z) * z
    y = y / np.linalg.norm(y)
    x = np.cross(y, z)
    result = np.eye(4, 4)
    result[0:3, 0] = x
    result[0:3, 1] = y
    result[0:3, 2] = z
    return result

plane = np.array([1., 1., 1., 1.])
normal = plane[0:3] / np.linalg.norm(plane[0:3])
mesh_box.transform(get_trans_mat(normal))

vis.draw_geometries([mesh_box, frame])

In [13]:
arrow = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=.02, cone_radius=.025, cylinder_height=.9, cone_height=.1)
arrow.transform(get_trans_mat(normal))
arrow.paint_uniform_color([.1, .7, .7])
frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
vis.draw_geometries([arrow, frame])